In [1]:
from langchain_core.prompts import PromptTemplate

s_template = PromptTemplate.from_template("Hello {name}")
s_template.invoke({"name": "Nikola"})

StringPromptValue(text='Hello Nikola')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

c_template = ChatPromptTemplate.from_messages([
    ("system", "Act as a sow expert"),
    ("user", "Please generate sow for client {client}")
])
c_template.invoke({"client": "Phiser"})

ChatPromptValue(messages=[SystemMessage(content='Act as a sow expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='Please generate sow for client Phiser', additional_kwargs={}, response_metadata={})])

In [34]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage

placeholder_template = ChatPromptTemplate.from_messages([
    ("system", "Act as sow expert for client {client}"),
    MessagesPlaceholder("my_msgs")
])

placeholder_template.invoke({"my_msgs": [HumanMessage(content="Create sow for me for client {client}")], "client": "foo"})

ChatPromptValue(messages=[SystemMessage(content='Act as sow expert for client foo', additional_kwargs={}, response_metadata={}), HumanMessage(content='Create sow for me for client {client}', additional_kwargs={}, response_metadata={})])

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [15]:
model.invoke([HumanMessage("Hi! I'm Bon")])

AIMessage(content='Hello Bon! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-af44d70c-2b75-4702-b5a2-9eef4512296d-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [16]:
model.invoke([HumanMessage("What is my name?")])

AIMessage(content="I'm sorry, I do not have the ability to know your name as I am an AI assistant.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 12, 'total_tokens': 33, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-290dcdfd-391c-4b0a-b458-0a22952a0840-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [17]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage("Hi! I'm Bon"),
    AIMessage("Hello Bon! How can I assist you today?"),
    HumanMessage("What is my name?")
])

AIMessage(content='Your name is Bon. How can I assist you today, Bon?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5dbc2ec-1b9b-427a-9aa2-9c3c9456b512-0', usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [46]:
from typing import Sequence

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    

# Define a new graph
workflow = StateGraph(state_schema=State)


# Define the function that calls the model
def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc123"}}

In [20]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
#output["messages"][-1].pretty_print()  # output contains all messages in state
output

{'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='c64c061b-b0f5-40c0-a899-d29e408b2eb2'),
  AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cff9095c-b25a-4bf5-98a4-e6f4a269b446-0', usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}

In [24]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [64]:
def converse(thread_id: str, message_in: str):
    messages_in = [HumanMessage(message_in)]
    for chunk, metadata in app.stream({
        "messages": messages_in,
        "language": "Spanish"
    }, {
        "configurable": {
            "thread_id": thread_id
        }
    },
    stream_mode="messages"                         
    ):
        print(f"chunk: {chunk}")
        print(f"metad: {metadata}")
        if isinstance(chunk, AIMessage):
            yield chunk.content
    #message_out: AIMessage = messages_out["messages"][-1]
    #return message_out.pretty_print()

In [65]:
for chunk in converse("user_a_chat_3", "What is my name?"):
    print(chunk)

chunk: content='What is my name?' additional_kwargs={} response_metadata={} id='72e91546-2167-41f2-aba7-937888809492'
metad: {'thread_id': 'user_a_chat_3', 'langgraph_step': 36, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_path': ('__pregel_pull', '__start__'), 'langgraph_checkpoint_ns': '__start__:8139dc4f-ebc2-a6a3-62f0-3d5eead7a76e'}
chunk: content='' additional_kwargs={} response_metadata={} id='run-a2127420-f9ca-4a5c-9801-b998fab184b4'
metad: {'thread_id': 'user_a_chat_3', 'langgraph_step': 37, 'langgraph_node': 'model', 'langgraph_triggers': ['start:model'], 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:3736d1c4-b35f-f088-d80c-55114248dcf5', 'checkpoint_ns': 'model:3736d1c4-b35f-f088-d80c-55114248dcf5', 'ls_provider': 'openai', 'ls_model_name': 'gpt-3.5-turbo', 'ls_model_type': 'chat', 'ls_temperature': 0.7}

chunk: content='T' additional_kwargs={} response_metadata={} id='run-a2127420-f9ca-4a5c-9801-b998fab184b4

In [45]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [35]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]